In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from sklearn.svm import SVC

### $\Large\textbf{Part(a)}$

$min_{\beta}   \ \lambda \| \beta \|_{2}^2 + \| y- X\beta \|_2^2 = min_{\beta} \ Z \ \  (say)$

let X is of order N x D

$Z = \lambda \| \beta \|_{2}^2 + \| y- X\beta \|_2^2 \\ \Rightarrow \lambda\beta^T \beta   + (y-X\beta)^T(y-X\beta)
\\ \Rightarrow y^T y - \beta ^T X^T y + \beta^T X ^T X \beta - y^T X \beta + \lambda \beta^T \beta$

To find the optimal weight vector $\boldsymbol{\beta}$, we take the derivative of the cost function with respect to $\boldsymbol{\beta}$ and set it to zero

$\Rightarrow 2X^T X \beta -2X^Ty +2\lambda \beta =0 \\ \Rightarrow X^TX\beta + \lambda \beta = X^Ty \ \  \ \ \ \  ....(1)$
 $ or X^TX\beta -X^Ty=-\lambda \beta  
\\ \Rightarrow \beta = \frac{1}{\lambda} X^T(y-X\beta) \\ so \  \beta \  can \  be  \ written \  as \\ \beta= X^T \alpha \ \ \ \   ...(2) \ where \ \ \alpha = \frac{1}{\lambda} (y-X\beta) \\ 
now \   we  \ can \  write \  (1) \  as \  
 \\ \beta=(X^T X + \lambda I_{D})^{-1} X^T y
 \\ \Rightarrow \beta = X^T(X X^T + \lambda I_{N})^{-1}y  \ \ \ ..(3) \  by \  matrix  \ inversion \  lemma \\
  \ comparing  \ (2)  \  and \  (3)  \ we \ got \ \\ \alpha = (X X^T + \lambda I_{N})^{-1}y \\ now  \ in \  this \  relation  \ X^TX  \ \text{can be effectively replace by a kernel matrix using the kernel function let k as kernel } \\ 
  \alpha = (K + \lambda I_N)^{-1}y
  \\ \text{then we can rewrite} \ \beta \ as \ 
  \\ \beta=X^T \alpha = \sum_{i=1}^{i=N} 
\alpha_i X_i \\   \text{This tells us that the solution vector is just a linear sum of the N training vectors. When we plug this in at test time to compute the predictive mean},\\ \text{let's compute for a sample} \  x̃  \\  \\  ỹ(x̃) = \beta^Tx̃ = \sum_{i=1}^{i=N} \alpha_i x_i^Tx̃ = \sum_{i=1}^{i=N} \alpha_i K(x̃,x_i)$

### $\Large\textbf{Part(b)}$

In [2]:
df = pd.read_csv('Data_Q2.csv')

In [3]:
df.head()

,Temperature,Humidity,Wind Speed,Flow,Consumption
0,5.578,93.00,0.082,0.185,5935.174070
1,15.510,64.38,0.085,0.133,6044.657863
2,15.730,64.21,0.084,0.152,6061.944778
3,15.620,65.22,0.083,0.145,6108.043217
4,15.450,67.69,0.083,0.189,6119.567827


### $\Large\textbf{Part(c)}$

In [4]:
sd = StandardScaler()
df_scaled = pd.DataFrame(sd.fit_transform(df), columns = df.columns)

In [5]:
df_scaled.head()

,Temperature,Humidity,Wind Speed,Flow,Consumption
0,-1.931878,1.404846,-0.641791,-0.664370,-3.450208
1,0.407654,-0.874474,-0.640417,-0.665724,-3.282442
2,0.459476,-0.888013,-0.640875,-0.665229,-3.255953
3,0.433565,-0.807575,-0.641333,-0.665412,-3.185315
4,0.393521,-0.610863,-0.641333,-0.664266,-3.167655


In [6]:
y = df_scaled['Consumption']
X = df_scaled.drop(['Consumption'], axis=1)

In [7]:
X.head()

,Temperature,Humidity,Wind Speed,Flow
0,-1.931878,1.404846,-0.641791,-0.664370
1,0.407654,-0.874474,-0.640417,-0.665724
2,0.459476,-0.888013,-0.640875,-0.665229
3,0.433565,-0.807575,-0.641333,-0.665412
4,0.393521,-0.610863,-0.641333,-0.664266


In [8]:
y.head()

0   -3.450208
1   -3.282442
2   -3.255953
3   -3.185315
4   -3.167655
Name: Consumption, dtype: float64

### $\Large\textbf{Part(d)}$

In [9]:
X_train, x_test, y_train,  y_test = train_test_split(X, y, random_state=104, test_size=0.2, shuffle=True)

*where X_train and y_train belongs to set T1(80% of the original dataset) and x_test and y_test belongs to set T2(20% of the original dataset)*

In [10]:
T1 = pd.DataFrame(X_train.copy())
T1.insert(4, 'Consumption', y_train)
T1['Consumption'].value_counts()

 0.779256    6
-0.156700    5
 0.003278    5
 0.796916    5
 0.637980    5
            ..
-0.810104    1
-1.852018    1
 0.716846    1
-0.839148    1
-0.677657    1
Name: Consumption, Length: 525, dtype: int64

In [11]:
T2 = pd.DataFrame(x_test.copy())
T2.insert(4, 'Consumption', y_test)
T2['Consumption'].value_counts()

 0.609536    3
 0.090533    3
 0.646809    3
 0.099814    2
 0.593831    2
            ..
 0.626873    1
-1.260423    1
 0.814575    1
 0.579288    1
-0.120786    1
Name: Consumption, Length: 179, dtype: int64

It is clear from the above result that T1 and T2 don't have similar counts in Consumption column.

### $\Large\textbf{Part(e)}$

In [12]:
pipeline = make_pipeline(KernelRidge(kernel='rbf'))
param_range = np.linspace(0.1,100,30)
train_scores, val_scores = validation_curve(estimator=pipeline,X=X_train, y=y_train,cv=5,n_jobs=-1,param_name='kernelridge__gamma', param_range=param_range)
#print('train scores:',train_scores)
#print('val scores:',val_scores)

print('Printing more details of scores for each alpha:')
for i in range(len(param_range)):
  print('alpha:', param_range[i])
  print('train scores:', train_scores[i])
  print('val scores:', val_scores[i])
  print('**************************')

Printing more details of scores for each alpha:
alpha: 0.1
train scores: [0.17445777 0.19089983 0.18473138 0.1523181  0.17574001]
val scores: [0.12064416 0.03828975 0.11118193 0.22128271 0.12438234]
**************************
alpha: 3.544827586206897
train scores: [0.45991592 0.49020863 0.47481266 0.45467279 0.45449288]
val scores: [0.15949097 0.06113524 0.10533498 0.26190812 0.22834725]
**************************
alpha: 6.989655172413793
train scores: [0.55296314 0.57693166 0.56060645 0.55241684 0.54188563]
val scores: [0.20935224 0.08958941 0.15161923 0.26410426 0.24883065]
**************************
alpha: 10.43448275862069
train scores: [0.60109208 0.6163382  0.60446175 0.59837107 0.58262403]
val scores: [0.21417169 0.10921437 0.16788563 0.27036072 0.25597302]
**************************
alpha: 13.879310344827587
train scores: [0.62903424 0.63775292 0.62950334 0.62427705 0.60687575]
val scores: [0.20545073 0.12373169 0.17301499 0.27105924 0.25545956]
**************************
alpha

In [13]:
avg_train_scores = np.mean(train_scores,axis=1)
avg_val_scores = np.mean(val_scores,axis=1)
print('average train scores :',avg_train_scores)
print('average val scores :',avg_val_scores)

average train scores : [0.17562942 0.46682058 0.55696074 0.60057743 0.62548866 0.64152421
 0.6528342  0.66141384 0.66829054 0.67402365 0.67893621 0.68322706
 0.68702671 0.69042602 0.69349154 0.69627397 0.69881313 0.70114107
 0.70328402 0.70526374 0.70709852 0.70880385 0.71039298 0.71187734
 0.71326683 0.71457012 0.71579478 0.71694755 0.71803436 0.71906052]
average val scores : [0.12315618 0.16324331 0.19269916 0.20352109 0.20574324 0.2042766
 0.20113681 0.1973573  0.19345421 0.18966141 0.18606992 0.18270366
 0.17955748 0.17661527 0.1738582  0.17126807 0.16882852 0.16652527
 0.16434596 0.16227986 0.16031764 0.15845104 0.15667275 0.15497622
 0.15335555 0.15180537 0.15032079 0.14889733 0.1475309  0.14621769]


In [14]:
best_alpha = param_range[np.argmax(avg_val_scores)]
print('best_hyper_parameter from 5 fold CV:',best_alpha)
kernel = KernelRidge(kernel='rbf',gamma=best_alpha)

kernel.fit(X_train, y_train)
y_pred_train = kernel.predict(X_train)
y_pred_test = kernel.predict(x_test)


best_hyper_parameter from 5 fold CV: 13.879310344827587


### $\Large\textbf{Part(f)}$

In [15]:
print("rmse_train:", sqrt(mean_squared_error(y_train, y_pred_train)))
print("r_2_train:",r2_score(y_train, y_pred_train))
print("rmse_test:", sqrt(mean_squared_error(y_test, y_pred_test)))
print('r_2_test',r2_score(y_test,y_pred_test))

rmse_train: 0.6140862315539782
r_2_train: 0.6207767986432727
rmse_test: 0.8877996925168121
r_2_test 0.2206201979680208


### $\Large\textbf{Part(g)}$

In [16]:
frame2 = pd.read_csv('Data_Q2.csv')

frame2.loc[frame2['Consumption'] <= 6500, 'Class'] = 1
frame2.loc[(frame2['Consumption'] > 6500) & (frame2['Consumption'] <= 7000), 'Class'] = 2
frame2.loc[(frame2['Consumption'] > 7000) & (frame2['Consumption'] <= 7500), 'Class'] = 3
frame2.loc[(frame2['Consumption'] > 7500) & (frame2['Consumption'] <= 8000), 'Class'] = 4
frame2.loc[(frame2['Consumption'] > 8000) & (frame2['Consumption'] <= 8500), 'Class'] = 5
frame2.loc[(frame2['Consumption'] > 8500) & (frame2['Consumption'] <= 9000), 'Class'] = 6
frame2.loc[frame2['Consumption'] > 9000 , 'Class'] = 7

In [17]:
frame2

,Temperature,Humidity,Wind Speed,Flow,Consumption,Class
0,5.578,93.00,0.082,0.185,5935.174070,1.0
1,15.510,64.38,0.085,0.133,6044.657863,1.0
2,15.730,64.21,0.084,0.152,6061.944778,1.0
3,15.620,65.22,0.083,0.145,6108.043217,1.0
4,15.450,67.69,0.083,0.189,6119.567827,1.0
...,...,...,...,...,...,...
995,17.330,42.24,4.917,31.540,9443.855422,7.0
996,7.010,76.40,4.920,65.890,9449.638554,7.0
997,14.810,82.30,4.913,0.159,9449.638554,7.0
998,12.090,77.40,0.073,0.104,9449.638554,7.0


### $\Large\textbf{Part(h)}$

In [18]:
# Get the list of unique class labels
class_labels = frame2['Class'].unique()

# Standardize each subset separately
for label in class_labels:
    subset = frame2[frame2['Class'] == label].copy()
    subset_mean = subset.drop('Class', axis=1).mean()
    subset_std = subset.drop('Class', axis=1).std()
    subset.loc[:, subset.columns != 'Class'] = (subset.loc[:, subset.columns != 'Class'] - subset_mean) / subset_std
    frame2.loc[frame2['Class'] == label, subset.columns] = subset

# Save the standardized dataframe
frame2.to_csv('frame2_standardized.csv', index=False)

In [19]:
frame2

,Temperature,Humidity,Wind Speed,Flow,Consumption,Class
0,-1.633251,1.510316,-0.491477,-0.353228,-1.860482,1.0
1,1.046563,-1.473479,0.561688,-0.398291,-1.259987,1.0
2,1.105922,-1.491202,0.210633,-0.381826,-1.165172,1.0
3,1.076242,-1.385904,-0.140422,-0.387892,-0.912332,1.0
4,1.030374,-1.128393,-0.140422,-0.349762,-0.849122,1.0
...,...,...,...,...,...,...
995,0.569285,-2.345601,1.577694,-0.112411,1.703543,7.0
996,-1.974412,0.414453,1.579064,0.550097,1.744723,7.0
997,-0.051851,0.891161,1.575868,-0.717656,1.744723,7.0
998,-0.722282,0.495251,-0.633634,-0.718717,1.744723,7.0


### $\Large\textbf{Part(i)}$

In [20]:
T3_X, T4_X, T3_y, T4_y = train_test_split(frame2.iloc[:,:-1],frame2.iloc[:,-1], test_size=0.2,stratify=frame2.iloc[:,-1], random_state=104)

In [21]:
print('Class 1 sample in T3_y:', len([label for label in T3_y if label==1]))
print('Class 2 sample in T3_y:', len([label for label in T3_y if label==2]))
print('Class 3 sample in T3_y:', len([label for label in T3_y if label==3]))
print('Class 4 sample in T3_y:', len([label for label in T3_y if label==4]))
print('Class 5 sample in T3_y:', len([label for label in T3_y if label==5]))
print('Class 6 sample in T3_y:', len([label for label in T3_y if label==6]))
print('Class 7 sample in T3_y:', len([label for label in T3_y if label==7]))

Class 1 sample in T3_y: 12
Class 2 sample in T3_y: 22
Class 3 sample in T3_y: 82
Class 4 sample in T3_y: 174
Class 5 sample in T3_y: 224
Class 6 sample in T3_y: 204
Class 7 sample in T3_y: 82


In [22]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state = 104)
T3_X_new, T3_y_new = oversample.fit_resample(T3_X, T3_y)

In [23]:
print('Class 1 sample in T3_y:', len([label for label in T3_y_new if label==1]))
print('Class 2 sample in T3_y:', len([label for label in T3_y_new if label==2]))
print('Class 3 sample in T3_y:', len([label for label in T3_y_new if label==3]))
print('Class 4 sample in T3_y:', len([label for label in T3_y_new if label==4]))
print('Class 5 sample in T3_y:', len([label for label in T3_y_new if label==5]))
print('Class 6 sample in T3_y:', len([label for label in T3_y_new if label==6]))
print('Class 7 sample in T3_y:', len([label for label in T3_y_new if label==7]))

Class 1 sample in T3_y: 224
Class 2 sample in T3_y: 224
Class 3 sample in T3_y: 224
Class 4 sample in T3_y: 224
Class 5 sample in T3_y: 224
Class 6 sample in T3_y: 224
Class 7 sample in T3_y: 224


In [24]:
pipeline = make_pipeline(SVC(kernel='rbf'))
param_range = np.linspace(0.1,10,20)
train_scores, val_scores = validation_curve(estimator=pipeline, X=T3_X_new.iloc[:,:-1], y=T3_y_new, cv=5,n_jobs=-1, param_name='svc__gamma', param_range=param_range)
#print('train scores:',train_scores)
#print('val scores:',val_scores)

print('Printing more details of scores for each alpha:')
for i in range(len(param_range)):
  print('alpha:', param_range[i])
  print('train scores:', train_scores[i])
  print('val scores:', val_scores[i])
  print('**************************')

Printing more details of scores for each alpha:
alpha: 0.1
train scores: [0.3907496  0.36363636 0.38516746 0.39521912 0.35856574]
val scores: [0.32484076 0.26433121 0.31528662 0.3514377  0.37060703]
**************************
alpha: 0.6210526315789474
train scores: [0.6523126  0.63955343 0.64832536 0.6310757  0.64063745]
val scores: [0.49681529 0.54458599 0.58917197 0.57188498 0.65814696]
**************************
alpha: 1.142105263157895
train scores: [0.74720893 0.74082935 0.72727273 0.7314741  0.73386454]
val scores: [0.57006369 0.63694268 0.64649682 0.62939297 0.72523962]
**************************
alpha: 1.6631578947368424
train scores: [0.78867624 0.79186603 0.76714514 0.77051793 0.77131474]
val scores: [0.61146497 0.66878981 0.67515924 0.66134185 0.72204473]
**************************
alpha: 2.18421052631579
train scores: [0.81419458 0.82057416 0.79904306 0.79840637 0.81115538]
val scores: [0.63694268 0.67834395 0.69426752 0.69648562 0.73801917]
**************************
alpha

In [25]:
avg_train_scores = np.mean(train_scores,axis=1)
avg_val_scores = np.mean(val_scores,axis=1)
print('average train scores :',avg_train_scores)
print('average val scores :',avg_val_scores)

average train scores : [0.37866766 0.64238091 0.73612993 0.77790401 0.80867471 0.82812597
 0.84980982 0.86495676 0.87516003 0.88440649 0.89333511 0.90178489
 0.90895925 0.9139019  0.91836634 0.9228304  0.92713573 0.92968718
 0.9328752  0.93702015]
average val scores : [0.32530067 0.57212104 0.64162715 0.66776012 0.68881179 0.69009178
 0.70156895 0.70730754 0.7085855  0.71432816 0.71688203 0.72197554
 0.72453145 0.72326163 0.72582162 0.72518671 0.72519078 0.72327791
 0.72327995 0.72391893]


In [26]:
best_alpha = param_range[np.argmax(avg_val_scores)]
print('best_hyper_parameter from 5 fold CV:',best_alpha)

best_hyper_parameter from 5 fold CV: 7.394736842105264


In [27]:
svc_kernel =SVC(kernel='rbf', gamma = best_alpha)

svc_kernel.fit(T3_X_new.iloc[:,:-1], T3_y_new)

y_pred_train =svc_kernel.predict(T3_X_new.iloc[:,:-1])
y_pred_test=svc_kernel.predict(T4_X.iloc[:,:-1])

In [28]:
train_score = svc_kernel.score(T3_X_new.iloc[:,:-1], T3_y_new)

test_score =svc_kernel.score(T4_X.iloc[:,:-1], T4_y)

print('Train accuracy:',train_score)
print('Test accuracy:',test_score)

Train accuracy: 0.9158163265306123
Test accuracy: 0.585


### $\Large\textbf{Part(j)}$

In [29]:
x_train=[[] for i in range(7)]
y_train=[[] for i in range(7)]
x_test=[[] for i in range(7)]
y_test=[[] for i in range(7)]
kernel={}
for i in range(7):
  x_train[i]=T3_X[T3_y==i+1].iloc[:,:-1]
  y_train[i]=T3_X[T3_y==i+1].iloc[:,-1]
  x_test[i]=T4_X[T4_y==i+1].iloc[:,:-1]
  y_test[i]=T4_X[T4_y==i+1].iloc[:,-1]
  pipeline = make_pipeline(KernelRidge(kernel='rbf'))
  param_range = np.linspace(0.1,10,20)
  train_scores, val_scores = validation_curve(estimator=pipeline, X=x_train[i], y=y_train[i], cv=5,n_jobs=-1, param_name='kernelridge__gamma', param_range=param_range)
  #print('\n \n \n for class',i+1)
  #print('train scores:',train_scores)
  #print('val scores:',val_scores)

  print('Printing more details of scores for each alpha:')
  for j in range(len(param_range)):
    print('alpha:', param_range[j])
    print('train scores:', train_scores[j])
    print('val scores:', val_scores[j])
    print('**************************')
  avg_train_scores = np.mean(train_scores,axis=1)
  avg_val_scores = np.mean(val_scores,axis=1)
  print('average train scores :',avg_train_scores)
  print('average val scores :',avg_val_scores)
  best_alpha = param_range[np.argmax(avg_val_scores)]
  print('best_hyper_parameter from 5 fold CV:',best_alpha)
  kernel[i+1] = KernelRidge(kernel='rbf',gamma=best_alpha)
  kernel[i+1].fit(x_train[i], y_train[i])


Printing more details of scores for each alpha:
alpha: 0.1
train scores: [0.47529665 0.45845749 0.44767129 0.403782   0.4990898 ]
val scores: [-0.02440624 -0.23930199  0.26949646  0.25097852 -0.02818372]
**************************
alpha: 0.6210526315789474
train scores: [0.80031409 0.78021753 0.76167428 0.75949411 0.77740419]
val scores: [0.10773436 0.24918868 0.8100334  0.3702986  0.28615916]
**************************
alpha: 1.142105263157895
train scores: [0.82713044 0.79025971 0.77311333 0.77847627 0.78224713]
val scores: [0.05643585 0.25274359 0.84990529 0.34067031 0.34410599]
**************************
alpha: 1.6631578947368424
train scores: [0.82929316 0.79012359 0.77241754 0.77938377 0.77934629]
val scores: [0.03311726 0.12386855 0.83457172 0.31737374 0.35635919]
**************************
alpha: 2.18421052631579
train scores: [0.82781096 0.78880469 0.77085813 0.77838184 0.77665096]
val scores: [0.02072177 0.00437217 0.80401109 0.29601076 0.35517881]
**************************


### $\Large\textbf{Part(k)}$

In [47]:
y_pred_train=[]
y_train=[]
label=svc_kernel.predict(T3_X.iloc[:,:-1])
for i in range(1,8):
  if sum(label==i)>0:
    y_pred_train.extend(list(kernel[i].predict(T3_X[label==i].iloc[:,:-1])))
    y_train.extend(list(T3_X[label==i].iloc[:,-1]))

In [51]:
print("root mean square error for training data t3 is ", sqrt(mean_squared_error(y_train, y_pred_train)))
print("R^2 for training data set(T3) is ",r2_score(y_train, y_pred_train))

root mean square error for training data t3 is  0.6907600198011522
R^2 for training data set(T3) is  0.5212620381866099


In [49]:
y_pred_test=[]
y_test=[]
label=svc_kernel.predict(T4_X.iloc[:,:-1])
for i in range(1,8):
  if sum(label==i)>0:
    y_pred_test.extend(list(kernel[i].predict(T4_X[label==i].iloc[:,:-1])))
    y_test.extend(list(T4_X[label==i].iloc[:,-1]))

In [52]:
print("root mean square error for test data t4 is ", sqrt(mean_squared_error(y_test, y_pred_test)))
print('R^2 for test dataset(T4) is',r2_score(y_test,y_pred_test))

root mean square error for test data t4 is  1.0164141428739681
R^2 for test dataset(T4) is -0.057070316187038506


### $\Large\textbf{Part(l)}$

* In part (f), r_2 value is higher while in part (k), rmse value is higher.

* The two-stage approach of classification-followed-by-regression is useful when the target variable has a complex relationship with the predictor variables, and a single regression model cannot adequately capture the relationship.
In such cases, the two-stage approach first classifies the data into different groups based on a categorical variable, and then fits separate regression models for each group. This approach can capture the unique relationships between the predictor variables and the target variable within each group, resulting in more accurate predictions.

* On the other hand, a simple regression approach on the full dataset assumes a single relationship between the predictor variables and the target variable, which may not be accurate for all observations in the dataset. This approach may result in a less accurate model that does not capture the nuances of the data.